In [11]:

from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler


In [12]:

# Initialize Spark session
spark = SparkSession.builder.appName("spark_sample_test").master("spark://spark-master:7077") \
        .getOrCreate()

In [13]:

# Sample data in a list format
data = [
    ("Alice", "Female", 29, 3000),
    ("Bob", "Male", 34, 4000),
    ("Charlie", "Female", 25, 3500),
    ("David", "Male", 40, 4500),
    ("Eve", "Female", 30, 3200)
]

# Define column names
columns = ["Name", "Gender", "Age", "Salary"]

# Create a DataFrame
df = spark.createDataFrame(data, columns)

# Show the original DataFrame
print("Original Data:")
df.show()

Original Data:


+-------+------+---+------+
|   Name|Gender|Age|Salary|
+-------+------+---+------+
|  Alice|Female| 29|  3000|
|    Bob|  Male| 34|  4000|
|Charlie|Female| 25|  3500|
|  David|  Male| 40|  4500|
|    Eve|Female| 30|  3200|
+-------+------+---+------+



In [14]:

# Step 1: Index the "Gender" column (convert categorical data to numerical)
indexer = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
df_indexed = indexer.fit(df).transform(df)

print("After StringIndexer (Gender -> GenderIndex):")
df_indexed.show()

After StringIndexer (Gender -> GenderIndex):
+-------+------+---+------+-----------+
|   Name|Gender|Age|Salary|GenderIndex|
+-------+------+---+------+-----------+
|  Alice|Female| 29|  3000|        0.0|
|    Bob|  Male| 34|  4000|        1.0|
|Charlie|Female| 25|  3500|        0.0|
|  David|  Male| 40|  4500|        1.0|
|    Eve|Female| 30|  3200|        0.0|
+-------+------+---+------+-----------+



In [15]:

# Step 2: One-hot encode the "GenderIndex" column
encoder = OneHotEncoder(inputCol="GenderIndex", outputCol="GenderVec")
df_encoded = encoder.fit(df_indexed).transform(df_indexed)

print("After OneHotEncoder (GenderIndex -> GenderVec):")
df_encoded.show()

After OneHotEncoder (GenderIndex -> GenderVec):
+-------+------+---+------+-----------+-------------+
|   Name|Gender|Age|Salary|GenderIndex|    GenderVec|
+-------+------+---+------+-----------+-------------+
|  Alice|Female| 29|  3000|        0.0|(1,[0],[1.0])|
|    Bob|  Male| 34|  4000|        1.0|    (1,[],[])|
|Charlie|Female| 25|  3500|        0.0|(1,[0],[1.0])|
|  David|  Male| 40|  4500|        1.0|    (1,[],[])|
|    Eve|Female| 30|  3200|        0.0|(1,[0],[1.0])|
+-------+------+---+------+-----------+-------------+



In [16]:
# Step 3: Assemble the features (Age, Salary, and GenderVec) into a single feature vector
assembler = VectorAssembler(
    inputCols=["Age", "Salary", "GenderVec"],  # Input features
    outputCol="features"  # Output column with assembled vector
)

df_final = assembler.transform(df_encoded)

print("After VectorAssembler (Assembled Features):")
df_final.select("Name", "features").show(truncate=False)

# Stop the Spark session
spark.stop()

After VectorAssembler (Assembled Features):
+-------+-----------------+
|Name   |features         |
+-------+-----------------+
|Alice  |[29.0,3000.0,1.0]|
|Bob    |[34.0,4000.0,0.0]|
|Charlie|[25.0,3500.0,1.0]|
|David  |[40.0,4500.0,0.0]|
|Eve    |[30.0,3200.0,1.0]|
+-------+-----------------+

